<a href="https://colab.research.google.com/github/OhDasom88/cnn_classifier/blob/Ryles_CNN/jiwon_cnn_0_8374.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lovit/soynlp.git


Cloning into 'soynlp'...
remote: Enumerating objects: 5042, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 5042 (delta 374), reused 512 (delta 320), pack-reused 4457
Receiving objects: 100% (5042/5042), 34.86 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (3413/3413), done.


In [2]:
!git clone https://github.com/ratsgo/embedding.git

Cloning into 'embedding'...
remote: Enumerating objects: 1736, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 1736 (delta 73), reused 68 (delta 33), pack-reused 1618
Receiving objects: 100% (1736/1736), 448.47 KiB | 10.68 MiB/s, done.
Resolving deltas: 100% (1158/1158), done.


In [3]:
!git clone https://github.com/lovit/soyspacing.git

Cloning into 'soyspacing'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 270 (delta 13), reused 33 (delta 12), pack-reused 235
Receiving objects: 100% (270/270), 2.14 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [4]:
#영화 데이터 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings_test.txt
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt

--2021-08-01 13:25:12--  https://github.com/e9t/nsmc/raw/master/ratings_test.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt [following]
--2021-08-01 13:25:13--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  28.5MB/s    in 0.2s    

2021-08-01 13:25:13 (28.5 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2021-08-01 13:25:13--  https://github.com/e9t/nsmc/raw/master/rating

In [128]:
#konlpy와 mecab 다운로
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy    
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcurl4 libcurl4-openssl-dev
Suggested packages:
  libcurl4-doc libidn11-dev libkrb5-dev libldap2-dev librtmp-dev
The following packages will be upgraded:
  curl libcurl4 libcurl4-openssl-dev
3 upgraded, 0 newly installed, 0 to remove and 90 not upgraded.
Need to get 679 kB of archives.
After 

In [56]:
import pandas as pd
import numpy as np
import os
import json
import nltk
from konlpy.tag import Okt
import tqdm
import re

In [7]:
#test, train df 선언
test_df=pd.read_csv("./ratings_test.txt", sep="\t").dropna()
train_df=pd.read_csv("./ratings_train.txt", sep="\t").dropna()

In [8]:
test_path="./ratings_test.txt"
train_path="./ratings_train.txt"

In [9]:
def load_data_and_label(path):
    data = []
    labels = []
    f = open(path, 'r')
    # 첫 줄을 날림.
    f.readline()
    for line in f.readlines():
        data.append(line.split('\t')[1].replace(" ", ""))
        labels.append(line.split('\t')[2][:-1])
    
    data = np.asarray(data)
    labels = np.asarray(labels)

    return data, labels

In [10]:
train_sentence, train_labels=load_data_and_label(train_path)

In [338]:
len(train_labels
    )

150000

In [12]:
import soyspacing

In [14]:
from soyspacing.soyspacing.countbase import CountSpace

corpus_fname="/content/ratings_train.txt"
model_fname="/content/drive/MyDrive/preprocessed/space-correct.model"
model=CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format=False)
model.load_model(model_fname,json_format=False)


all tags length = 139793 --> 139705, (num_doc = 150000)

In [97]:
#띄어쓰기 교정, 특수문자 제거

def sentence_making(document):
  new_doc=[]
  for doc in document:
   #doc을 ","로 스플릿된 리스트로 변환
      
      doc=doc.split(" ")
      
      new_model=[]
      for d in doc:
        d = re.sub(r"[^ 가-힣a-zA-Z0-9]"," ",d)
        new_model.extend(str(model.correct(d)[0]).split(" "))
      new_model=" ".join(new_model) 
      new_doc.append(new_model)
  return new_doc
  

In [100]:
train_cleaned=sentence_making(train_df["document"])
test_cleaned=sentence_making(test_df["document"])
      

In [103]:
new_doc_pd=pd.DataFrame(train_cleaned)
new_test_pd=pd.DataFrame(test_cleaned)

In [339]:
len(new_doc_pd)

149995

In [112]:
new_doc_pd.to_csv("/content/drive/MyDrive/preprocessed/new_spacing_doc", header=0)
new_test_pd.to_csv("/content/drive/MyDrive/preprocessed/new_spacing_test,", header=0)

In [294]:
new_doc_pd=pd.read_csv("/content/drive/MyDrive/preprocessed/new_spacing_doc",names=["id","doc"])
new_test_pd=pd.read_csv("/content/drive/MyDrive/preprocessed/new_spacing_test",names=["id","doc"])



In [248]:
new_doc_pd

,id,doc
0,0,아 더빙 진짜 짜증나네요 목소리
1,1,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나
2,2,너무 재밓었다그래서 보는것을 추천한다
3,3,교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4,4,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...,...
149990,149990,인간이 문제지 소는 뭔죄인가
149991,149991,평점이 너무 낮아서
149992,149992,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149993,149993,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상


In [242]:
from konlpy.tag import Mecab, Hannanum, Komoran

In [243]:
mecab=Mecab()
han=Hannanum()
Kom=Komoran()

In [288]:
#전처리 - 포스태깅 후 명사, 부사, 동사만
def preprocessing(text,mecab):
  new_text=[]
  for word, pos in mecab.pos(text):
    
    if pos[0] in ["N","V","M"] :
      
        new_text.append(word)
    else:
      pass

  return new_text

In [290]:
clean_train_text=[]
for text in tqdm.tqdm(new_doc_pd["doc"]):
    try:
        clean_train_text.append(preprocessing(text, mecab))
    except:
        clean_train_text.append([])

100%|██████████| 149995/149995 [00:16<00:00, 9045.55it/s]


In [295]:
clean_test_text=[]
for text in tqdm.tqdm(new_test_pd["doc"]):
    try:
        clean_test_text.append(preprocessing(text, mecab))
    except:
        clean_test_text.append([])

100%|██████████| 49998/49998 [00:05<00:00, 9352.43it/s]


In [350]:
import datetime
from tensorflow.python.keras.layers.core import Flatten

from tensorflow.python.keras.layers.pooling import MaxPool2D
start_time = datetime.datetime.now()
print(str(start_time))

from konlpy.tag import Mecab
import smart_open   
smart_open.open = smart_open.smart_open
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Embedding, BatchNormalization
from tensorflow.keras import datasets, layers
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# filename = '../movie_review/nsmc/ratings_train.txt'
# datastore = pd.read_csv(filename, delimiter='\t')
# filename = '../movie_review/nsmc/ratings_test.txt'
# test_data = pd.read_csv(filename, delimiter='\t')

# datastore = pd.read_csv("/new_doc",names=["col1","col2"], header=0)
# test_data = pd.read_csv('/new_test',names=["col1","col2"], header=0)


# sentences = clean_train_text
# labels = train_df["label"]
# # ids = datastore.col1.values

# test_sentences = clean_test_text
# test_labels = test_df["label"]
# # test_ids = test_data.col1.values


sentences = clean_train_text
labels = train_df["label"]
# ids = datastore.col1.values

test_sentences =clean_test_text
test_labels = test_df["label"]
# test_ids = test_data.col1.values


2021-08-02 01:29:26.104684


In [349]:
len(clean_train_text)

149995

In [351]:
type(sentences)

list

In [352]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<oov>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type)


In [353]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
sequences_test = tokenizer.texts_to_sequences(test_sentences)
padded = pad_sequences(sequences, maxlen=int(np.quantile(np.array([len(seqs) for seqs in sequences]), 0.9)), padding='post', truncating='post')
padded_test = pad_sequences(sequences_test, maxlen=padded.shape[-1], padding='post', truncating='post')
padded = np.dstack([padded]*2)
padded_test = np.dstack([padded_test]*2)

vocab_size = len(word_index)
embedding_dim = 100
max_length = padded.shape[-2]
channel_num = 2


In [355]:

# padded = padded.reshape(len(padded),channel_num,-1)
# padded_test = padded_test.reshape(len(padded),channel_num,-1)
padded = np.transpose(padded, (0, 2, 1))
padded_test = np.transpose(padded_test, (0, 2,1))

input1 = Input(shape=(channel_num, max_length))#일단 채널을 전달하고
x1 = Embedding(vocab_size+1, embedding_dim)(input1)

# x2 = tf.keras.layers.Reshape((max_length, embedding_dim,channel_num))(x1)
x2 = tf.transpose(x1, perm=[0,2,3,1])
x3 = []#여러개의 filter의 대표값을 저장할 공간

for i in range(2,6,1):
    #각 그램별 필터는 하나로 계산
    x31 = layers.Conv2D(filters=8, kernel_size=(i,embedding_dim), activation='relu', kernel_initializer='glorot_normal')(x2)
    x41 = MaxPool2D(pool_size=(x31.shape[1],1))(x31)
    x51 = Flatten()(x41)# 각 sample 별로 질문
    x3.append(x51)
merged = tf.concat(x3, axis=1)
x6 = Dense(64, kernel_initializer='glorot_normal')(merged)
x = layers.Dropout(0.5)(x6)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=input1, outputs=outputs)

print(model.summary())

learning_rate = 0.001
batch_size = 100
lr_decay = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate, 
                                                        len(labels)/batch_size*5, 
                                                        decay_rate=0.5, 
                                                        staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decay)
model.compile(optimizer= optimizer, loss= binary_crossentropy, metrics=[binary_accuracy])
[]

MODEL_SAVE_FOLDER_PATH = '../trained_model'
model_file_path = f'{MODEL_SAVE_FOLDER_PATH}/review_cnn-{{epoch:d}}-{{val_loss:.5f}}-{{val_binary_accuracy:.5f}}.hdf5'
cb_model_check_point = ModelCheckpoint(filepath=model_file_path, monitor='val_binary_accuracy', verbose=1, save_best_only=True)
cb_early_stopping = EarlyStopping(monitor='val_loss', patience=6)

model.fit(padded, np.array(labels).reshape(-1,1), epochs=10, validation_split=0.1, batch_size=32
, callbacks=[cb_model_check_point, cb_early_stopping]
)


pred = model.predict(padded_test)
print(pred)
score = model.evaluate(padded_test,  np.array(test_labels).reshape(-1,1))
print(score)
# try:
#     model = Word2Vec.load("./word2vec.model")
# except:
#     try:
#         with open('./test_corpus.npy', 'rb') as f:
#             corpus= np.load(f, allow_pickle=True)
#         print(datetime.datetime.now()-start_time)
#     except:
#         filename = './classifing_movie_review_kr/nsmc/ratings_train.txt'
#         data = pd.read_csv(filename, delimiter='\t')
#         print(datetime.datetime.now()-start_time)
#         mecab = Mecab(r'C:\mecab\mecab-ko-dic')#210625 되는지 확인해보기
#         data['document'].fillna('', inplace=True)
#         corpus = [mecab.morphs(para) for para in data['document'][:100000]]
#         print(datetime.datetime.now()-start_time)

#         with open('./test_corpus.npy', 'wb') as f:
#             np.save(f, np.array(corpus, dtype=object))
#         print(datetime.datetime.now()-start_time)

#     model = Word2Vec(sentences=corpus, size=100, window=5, min_count=1, workers=4)
#     print(datetime.datetime.now()-start_time)

#     model.save("./word2vec.model")

# print(model)
# # vector = model.wv['금리']  # get numpy vector of a word
# # sims = model.wv.most_similar('금리', topn=10)  # get other similar words
# # print(vector)
# # print(sims)


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 2, 21)]      0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 2, 21, 100)   4271400     input_10[0][0]                   
__________________________________________________________________________________________________
tf.compat.v1.transpose_9 (TFOpL (None, 21, 100, 2)   0           embedding_9[0][0]                
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 20, 1, 8)     3208        tf.compat.v1.transpose_9[0][0]   
____________________________________________________________________________________________

ValueError: ignored